# (Larsen et al., 2023) Statistical Challenges in Online Controlled Experiments: A Review of A/B Testing Methodology

Taking notes from this [Paper](https://arxiv.org/pdf/2212.11366.pdf). I try my best to be brief.

# Abstract

We review challenges that require new statistical methodologies to address them. In particular, we discuss the practice and culture of online experimentation as well as its statistics literature, placing the current methodologies within their relevant statistical lineages and providing illustrative examples of OCE applications

# 1 Introduction

## 1.1 Background

Organizations that have accepted OCEs as standard practice generally adopt a so-called "culture of experimentation", which is rooted in 3 tenets (Kohavi et al., 2013):
1. the organization wants to make data-driven decisions
2. the organization is willing to invest in the people and infrastructure needed to run trustworthy experiments
3. the organization recognizes that it is poor at assessing the value of ideas

The purpose of the paper is to review the statistical methodology associated with OCEs, summarize its accompanying literature, and provide an overview of open statistical problems.

## 1.2 The General Framework

Key notation that will be used throughout this review and basic statistical framework for OCEs.

Let $K$ be the number of variants. Since there is usually one control and one treatment, in most cases $K=2$.

For $K > 2$, determining which bucket is optimal typically reduces to a pairwise comparison between each treatment and the control.

$n$ experimental units (e.g., users, cookies, sessions, etc.,) are typically randomized in real time to one of the variants, and a response observation $Y_i$ is collected for each $i=1,...,n$.

Interest lies in optimizing some *metric*, which is a numerical summary of the response. Metrics are often, but not always, averages, although in some contexts, quantile or double-average metrics may be more suitable.

When the metric is an average, the primary goal of the experiment is to estimate the *average treatment effect* (ATE); the difference between the average outcome when the treatment is applied globally and when the control is applied globally.

Within the potential outcomes framework (Rubin, 1974), $Y_i(0)$ represents unit $i$'s response in the hypothetical scenario where $i$ receives control, and $Y_i(1)$ is the potential response when unit $i$ receives the treatment.

Letting $W_i$ denote the binary treatment indicator for unit $i$, and given a particular treatment assignment to all experimental units $\mathbb{W}=(W_1,...,W_n)'$, the expected outcome is $\mathbb{E}[\frac{1}{n}\sum_{i=1}^{n}Y_i(W_i)]=\mu(\mathbb{W})$, and the ATE is therefore given by

$$\begin{align}\tau&=\mu(1)-\mu(0)\\ &=\frac{1}{n}\sum_{i=1}^{n}\mathbb{E}[Y_i(1)-Y_i(0)]\end{align}$$

In reality, since $i$ can only be assigned to a single variant at a time, thus one cannot directly observe both potential outcomes, the ATE is typically estimated with the difference-of-group-means estimator,

$$\hat\tau=\frac{1}{n_1}\sum_{\{i:W_i=1\}}Y_i-\frac{1}{n_0}\sum_{\{i:W_i=0\}}Y_i$$

where $n_0$ and $n_1$ are respectively the sizes of the control and treatment groups such that $n_0+n_1=n$.

In practice it is also common to define the treatment effect as a *relative* percent, often referred to as *lift*, since it is easier to interpret and is more stable (over experiment duration, for example).

> Of-course, the above estimator is only unbiased under exchangeability, which there is for an OCE.

Statistical significance is the most common mechanism by which a given treatment's effectiveness is affirmed in an A/B test. Analyses of A/B tests are therefore most often carried out via two-sample hypothesis tests for $\tau$ with standard test statistics of the form $\hat\tau / \hat\sigma_{\hat\tau}$

Such analyses, and the designs that generate data for them, commonly assume that the response of each user does not depend on others' treatment assignments (SUTVA). SUTVA is a reasonable assumption for many scenarios, and Section 6 discusses violations.

In many scenarios, sample sizes are large enough to confidently exploit the central limit theorem, permitting the use of the standard normal null distribution. There are, however, scenarios in which only a fraction of the user base is experimented on and asymptotic normality cannot be assumed. Such scenarios are discussed in 2.2.

Given the heavy reliance on $p$-values it is important to acknowledge that the reproducibility crisis stemming from the misuse of hypothesis tests also plagues OCEs; $p$-value misinterpretation and problematic practices regularly lead to increased false-positive rates.

## 1.3 Roadmap

- Section 2: techniques for improving experimental power - a critical issue despite the relatively large sample sizes in OCEs
- Section 3/4: present literature regarding the challenges of estimating heterogeneous and long-term treatment effects
- Section 5: discusses the problem of optional stopping and how sequential testing methods have been adapted to run online experiments
- Section 6: All these sections assumes SUTVA, and here we summarize the literature that explores violations of this assumption
- Section 7: A call-to-action for further collaboration between academia and OCE practicioners

# 2 Sensitivity and Small Treatment Effects

The sentiment that OCEs do not suffer from inadequate sample sizes is misconceived (Tang et al., 2010) because for organizations that are larger (because they have more users), even small delta's are impactful, thus they would be interested in extremely small delta's, meaning that the effect of having large sample sizes can feel like they "cancel out".

Instead of the naive solution of extending the duration of the experiment, it is better practice to employ a tactic that is tailored to the reason for insufficient power, which is generally one of 3 causes.
1. The treatment impacts the entire user population and the effect is roughly homogeneous, but very small in magnitude. (addressed in 2.1)
2. Many experiments test features that do not affect all users, making the treatment effect highly attenuated (weakened) (addressed in 2.2)
3. The treatment effects on known subpopulations of interest, where sample sizes are smaller by definition (addressed in 3)

In general, research regarding improving experimental power for OCEs tend to focus on boosting *sensitivity*, either by directly reducing the variance of $Y_i$ or by reducing the variance of estimators for $\tau$.

## 2.1 Transforming $Y$, Method of Control Variates, and Stratified Sampling

In order to improve sensitivity, a common approach is to transform $Y$ into $Y^*$ of lower variance which, all else being equal, translates to a lower variance estimator of $\tau$.

Due to many kinds of metrics, it is not possible to identify a transofmration that is "one size fits all".

Kohavi et al. (2014) discusses the benefits of identifying skewed metrics and capping (truncating) them to improve sensitivity.

Other transformations for improving the sensitivity of $Y$ include binarizing count metrics and revenue.

Deng and Shi define *directionality* (consistent behavior in one direction for positive treatment effects and in the opposite direction for negative effects) as an important feature when choosing metrics, suggesting that one should leverage prior experiments to compile a corpus of good metrics and to evaluate sensitivity and directionality with Bayesian priors.

Deng and Shi (2016) also propose aggregating metrics in the form of a weighted linear combination, adopted and expanded upon in Kharitonov et al. (2017).

In Drutsa et al. (2015a), features are extracted from data while the experiment is running and used to forecast metrics over a hypothetical post-experiment period. This methodology may be applied to long-term effect estimation using statistical surrogacy, discussed further in Section 4.

In addition to transformations of $Y$, a popular approach is to define an efficient, mean-zero augmented estimator of $\tau$ using the method of control variates (Courthoud, 2022)

Briefly, this method assumes, in addition to i.i.d $\{Y_i\}_{i=1}^{n}$, the availability of independent observations of a covariate, $\{X_i\}_{i=1}^{n}$, such that $\mathbb{E}[X_i]=\mu_x$.

Often, these covariate measurements are collected from prior logs or experiments.

Let $Y_i^*=Y_i-\theta (X_i-\mu_x)$, then

$Var(Y_i^*)=Var(Y_i)+\theta^2Var(X_i)-2\theta Cov(Y_i,X_i)$

is minimized with respect to $\theta$ using the OLS solution

$$Cov(Y_i,X_i)/Var(X_i)$$

Putting this together in the context of sample means gives

$$Var(\bar Y^*)=(1-\rho^2)Var(\bar Y)\leq Var(\bar Y)$$

where $\rho=Corr(Y_i,X_i)$. Thus, an ATE estimator that uses the difference of treatment and control means of $Y_i^*$ tends to have lower variance than the traditional $\hat\tau$, particularly when $X_i$ is strongly correlated with $Y_i$. For OCE's, this technique is referred to as CUPED (Controlled Experiments Utilizing Pre-Experiment Data) and was first proposed by Deng et al. (2013).

The authors empirically demonstrate that an effective covariate choice is the same variable $Y_i$ but collected during a pre-experiment period ($X_i\equiv Y_i^{\text{pre}}$). Such a choice can drastically increase sensitivity and thereby reduce time to statistical significance in determining $H_1:\tau\neq 0$. The authors also demonstrate that $\mu_x$ need not be known when $X_i$ is uncorrelated with $W_i$ and they also emphasize that despite resembling ANCOVA, CUPED does not require any linear model assumptions and can be treated as efficiency augmentation as in semi-parametric estimation (Tsiatis, 2006).

A key open question with respect to CUPED applications concern the situation when the covariate alone is not sufficienctly correlated with the response. An approach that shows promise employes synthetic controls, where one identifies a similar population without treatment exposure to use as covariate modeling $Y$ (Zhang et al., 2021). Another technique is to take advantage of a phenomenom that occurs in online experimentation known as "triggering" (Deng et al., 2023), discussed in 2.2.

Deng et al. (2013) note that CUPED also permits nonlinear adjustments to the response variable. Following this, Poyarkov et al. (2016) develop an approach that assumes each user has a response $Y$ and a set of features $\textbf{F}\in\mathbb{R}^p$ independent of treatment assignment. Let $Y=f(\textbf{F})$, where $f$ is an unknown, non-parametric function estimated with ML. The covariate is chosen to be the predicted outcomes of $\hat f$. Poyarkov et al. (2016) then use $Y^*=Y-\hat f(\textbf{F})$ as the primary metric for estimating the ATE, noting an increase in sensitivity compared to traditional A/B tests.

Finally, a closely related to CUPED is stratified sampling, discussed in Section SM1.

## 2.2 Triggered Analysis

Sensitivity may be increased by analyzing only the users who could have been impacted by the experiment; those that were triggered into the analysis. Although this reduces sample size, the treatment effect among the triggered users is undiluted and therefore higher and easier to detect.

*Triggered analysis* broadly refers to an OCE analysis where one only considers users who have the potential of being impacted by an experiment, excluding those who would not be affected by the proposed variant. Users are said to have *triggered* the experiment when they exhibit behavior that results in direct exposure to their assigned variant.

Key analysis challenges include:
1. generalizing the results from the triggered users to a broader population
2. reducing the variance of $\tau$ estimators to offset the smaller sample sizes that result from triggering

For an in-depth discussion of triggering case-studies, see chapter 20 of TOCE. The following provides a brief description and an overview of research in this area.

Let $\Omega$ be the overall user poulation and $\Theta\subset\Omega$ the population of users who could be affected by the treatment. A given user is determined to belong to $\Theta$ via techniques such as conditional checks or counterfactual logging (Kohavi et al., 2020).

If $\Theta$ comprises only a modest fraction of $\Omega$, (i.e., $\frac{|\Theta|}{|\Omega|}\leq 0.2$, for instance), an experiment that samples data from the entire population could be severely under-powered, particularly when effect sizes are small (Kohavi et al., 2009). To mitigate this issue, practicioners focus analysis only on triggered users.

The difference-of-means estimator $\hat\tau_\Theta$ is an unbiased estimator for the ATE of the triggered population, $\tau_\Theta$, under standard assumptions.

However, $\tau_\Theta$ is typically larger than the population-level $\tau_\Omega$ and the corresponding estimator generally has greater variance. The process of estimating $\tau_\Omega$ with $\hat\tau_\Theta$ is referred to as estimating the *diluated treatment effect*. Below we describe methodological development in this area.

Most triggered analyses fall under the following framework. Assume a (not necessarily random) sample of $N$ units, $n$ of which are triggered. Each user $i$ interacts with the website on multiple separate events. During each event, $i$ may or may not trigger. The most common analysis technique is *user-trigger analysis*, which incorporates all events beginning with the first event where $i$ triggered. Such analyses are quite popular as they do not require any assumptions regarding the treatment effect, and are amenable to common user-level metrics.



# 3. Heterogeneous Treatment Effects


Treatment effects on subgroups that differ from the population-level ATE are known as *heterogeneous treatment effects (HTE)* and are commonly of interest in OCEs.

Three key challenges are associated with estimating HTEs:
1. small treatment effects often make online studies under-powered, resulting in high false negative rates for subgroup effects
2. testing for HTEs tend to risk inflated Type I error rates due to multiple comparisons
3. in cases where users are not randomized to the subgroups under comparison, the usual tension between correlation and causation manifests.

Below we review existing methodologies that are commonly used in the context of OCEs to address this problem.

Assume each unit $i$ has a pair of potential outcomes $\{Y_i(1),Y_i(0)\}$ and a vector of pre-treatment covariates $X_i$, with $e(x)=Pr(W_i=1|X_i=x)$ being the probability that a user is treated given a particular value of the covariate. For randomized studies where causality may be inferred, $e(X_i)$ is known and technically independent of $X_i$.

Most of the literature regarding HTEs employs the following assumptions:
1. SUTVA
2. *unconfoundedness* ($\{Y_i(1),Y_i(0)\}\perp\!\!\!\perp W_i|X_i$)

The main goal is to estimate the *conditional average treatment effect (CATE)*,

$$\tau(x)=\mathbb{E}[Y_i(1)-Y_i(0)|X_i=x]$$

Another key challenge is to detect exactly for which specific levels of the covariate $\tau(x)$ differs from $\tau$ and, given several covariates, identifying which $X$'s are the source of heterogeneity.

Interpretation is crucial in the online industry, thus a popular approach is to assume a linear mapping from $Y_i$ to $(W_i,X_i)$ from which main and interaction effects may be estimated.

Unsurprisingly, the relationship between $Y_i$ and $X_i$ is often highly complex, thus a common method is to use the semi-parametric model from Robinson (1988),

$$Y_i=\tau(X_i)W_i+g(X_i)+\epsilon_i$$

which makes no assumption about the forms of $\tau(X_i)$ and $g(X_i)$.

HTE esitmation is a ripe target for machine learning methods and researcherse have approached this problem using a technique called "Double Machine Learning" (DML) Chernozhukov et al., 2017).

Other popular machine methods for estimating $\tau(X)$ are regression trees and random forests.

While causal trees and forests can be used to estimate the HTE (Wager and Athey, 2018) and they do not require linearity of treatment effects (and perhaps more conceptually intuitve than DML), they are somewhat lacking in terms of interpretability compared to effect estimates from DML.



# 4. Long-Term Effects

ATE $\tau$ should not be viewed as a constant with respect to time $(t)$, it should more appropriately be regarded as a function of it: $\tau(t)$.

Current OCE literature regarding long-term effect estimation is highly context-specific. We begin by introducing several distinct approaches that draw from a variety of statistical fields, and then finish with discussion of one area in particular that shows promise in providing a statistical framework for modeling and estimating long-term effects in online settings.

A straightforward way to assess long-term effects is to simply run the experiment longer and ensure that the appropriate metrics for capturing long-term behavior are observed. However, there are many pitfalls with running long-term controlled experiments for the purpose of estimating long-term effects (Kohavi et al., 2009).

In Wang et a. (2019), long-term effects are characterized as a form of bias due to heterogeneous treatment effects. In this context, long-term effects manifest because heavy-users tend to be included in experiments at higher rates than light-users, biasing the ATE particularly in the short-term. Here, the treatment effect is presumed to be different depending on whether user $i$ is a heavy- or light- user.

Another common solution is to define and measure short-term *driver metrics* that are causally linked to the long-term effect (Kohavi et al., 2020). This allows focusing on experiments on short-term goals while still taking into account the long-term effects. It is noted that using query-based models instead of click-based models tend to serve as better proxies of long-term metrics. Google implemented a "learned click-through-rate" as a driver metric for estimating long-term effect on revenue using a linear function of short-term revenue and the estimated learned click-through-rates. The model has been successfully deployed by Google and is widely cited in the OCE literature.

A recent paper by Sadeghi et al. (2022) proposes an observational approach based on difference-in-differences to estimate user learning and hence the long-term treatment effect in contexts where novelty and primacy effects exist.  



# 5. Optional Stopping

Methods that support early termination without compromising overall statistical validity are desirable.

While it is well known that the practice of "peeking" seriously inflates false positive rates (Johari et al., 2022a), there are nevertheless situations where having a mechanism for *optional stopping* is desirable.

Below we describe the development and application of methods in this area of literature.

As per the field of sequential testing, interest lies in assessing $H_0:\tau=0$ using sample-size-dependent decision rules. Within this class of methods, Type I error is controlled at each current sample size $n$, which avoids the inflated risk of Type I error that is associated with preemptively stopping an experiment when the current $p$-value is statistically significant by chance. While such methods improve testing efficiency due to the lower sample size as a sequential test will terminate at on average, existing methodology is not well-suited for all OCE applications, such as monitoring multiple metrics (e.g., the OEC and guardrails). Additionally, the reduced sample sizes guarantee under-powered HTE inference across user segments.

The following section broadly introduces the method of sequential testing as it pertains to ongoing evaluation of the treatment effects of interest in OCEs.

The majority of the OCE literature in sequential testing builds on the classic *sequential probability ratio test* (SPRT) developed by Walk (1945).

The first and perhaps most widely-known application of sequential testing in OCEs is a modified version of SPRT called the *mixture sequential probability ratio test*, or mSPRT (Johari et al., 2022a). This is currently developed by Optimizely. However, Johari et al. (2022a) derive their optimality conditions for mSPRT only for data that comes from the exponential family of distributions, which does not include distributions for the ratio metrics that are popular in industry. Theer are also assumptions and limitations (not mentioned in these notes, sorry).

There are also many other literature on various different methods.

# 6. Interference

The violation of SUTVA is referred to as *interference*, *spillover*, or *leakage*. There are typically two different forms of interference:
1. *network* interference, arises when the units are connected to one another through a network such as a social network
2. *marketplace* interference, arises when units compete for shared resources in two-sided or three-sided marketplaces.

Tradtional randomization no longer adequately approximates the counterfactuals and hence standard difference-in-means estimators are no longer unbiased. A rich literature has recently been developed that carefully considers both the design and analysis of OCEs in the presence of interference, and analysis methods that provide unbiased treatment effect estimates in the presence of interference. (many ideas actually come from the clinical RCT literature, such as cluster randomized trials and crossover designs).

In network A/B tests, cluster-based randomization have gained widespread attention, which involve first partitioning the network into mostly disjoint clusters, commonly via community detection algorithms. Randomization is then performed at the cluster level. However, with the clusters being the randomization unit (instead of users), the effective sample size and hence power is dramatically reduced. Saint-Jacques et al. (2019) therefore propose the use of many smaller *ego-clusters*.

In marketplace interference, *switchback* experiments are commonly used to sequentially alternate units between treatment and control over time (Bojinov et al., 2020).

In marketplaces based on auctions, interference due to shared resources is also a problem. This leads to "cannibalization bias", whereby the margin by which the treatment looks better than control is exaggerated, because when treatment wins, the control must lose. Liu et al. (2021b) propose *budget-split* designs that eliminate the opportunity for cannibalization bias by splitting the available resources (i.e., the budget) equally and independently between treatment and control.

There are also methods to *model* the interference rather than eliminating it. Of-course, accurately modeling interference is non-trivial.